In [1]:
%pip install -q tensorflow

Note: you may need to restart the kernel to use updated packages.


## Step 1: Load and Process Data

To demonstrate how unsupervised learning can be used for anomaly detection, we will use the MNIST dataset. We will take all the 9s and add a few 1s (which are the anomalous data points) and see if the model is able to detect them.

In [2]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

2025-04-18 19:58:06.722871: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-18 19:58:07.095714: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-18 19:58:07.208797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745006287.387938    5330 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745006287.442026    5330 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745006288.040094    5330 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [3]:
#Normalize data between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

x_train_flat = x_train.reshape(-1, 28 * 28)
x_test_flat = x_test.reshape(-1, 28 * 28)

In [4]:
import numpy as np
from sklearn.utils import shuffle

# Filter all 9's and a few 1's from the training dataset
nines_train = x_train_flat[y_train == 9]
ones_train = x_train_flat[y_train == 1][:100]  # Take the first 100 ones (adjust as needed)
x_train_anomaly = np.vstack([nines_train, ones_train])
y_train_anomaly = np.hstack([np.zeros(len(nines_train)), np.ones(len(ones_train))])  # 0 for normal, 1 for anomaly

# Filter all 9's and a few 1's from the testing dataset
nines_test = x_test_flat[y_test == 9]
ones_test = x_test_flat[y_test == 1][:20]  # Take the first 20 ones (adjust as needed)
x_test_anomaly = np.vstack([nines_test, ones_test])
y_test_anomaly = np.hstack([np.zeros(len(nines_test)), np.ones(len(ones_test))])  # 0 for normal, 1 for anomaly

# Shuffle the datasets to mix the anomalies with normal data
x_train_anomaly, y_train_anomaly = shuffle(x_train_anomaly, y_train_anomaly, random_state=42)
x_test_anomaly, y_test_anomaly = shuffle(x_test_anomaly, y_test_anomaly, random_state=42)

In [5]:
print(x_train_anomaly.shape)

(6049, 784)


## Step 2: Train the Model

Unlike supervised learning, there are expected to be no labels in unsupervised learning. Therefore, only the flattened image data will be used for the clustering model. The way clustering works is that it chooses n points in the feature space, and then tries to minimize the distance between each point and the surrounding data points.

In [6]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=1, random_state=42)
kmeans.fit(x_train_anomaly)

KMeans(n_clusters=1, random_state=42)

In [7]:
test_pred = kmeans.predict(x_test_anomaly)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Find indices of predicted anomalies
anomalous_indices = np.where(test_pred == 1)[0]

# Plot a few anomalies
num_anomalies_to_plot = 10  # Adjust the number of anomalies to visualize
plt.figure(figsize=(10, 10))
for i, idx in enumerate(anomalous_indices[:num_anomalies_to_plot]):
    plt.subplot(1, num_anomalies_to_plot, i + 1)
    plt.imshow(x_test_anomaly[idx].reshape(28, 28), cmap='gray')
    plt.axis('off')
    plt.title("Anomaly")
plt.show()

<Figure size 1000x1000 with 0 Axes>